# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [24]:
from aimakerspace.text_utils import UniversalFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [25]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [26]:
text_loader = UniversalFileLoader("data")
documents = text_loader.load()
len(documents)

6

In [27]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [28]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

428

Let's take a look at some of the documents we've managed to split.

In [29]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [30]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [31]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [32]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

In [33]:
vector_db.search_by_text("Where is Amatol located?", k=3)

[(' the population of Amatol had virtually van-ished, with only a few hundred remaining. /T_he former munitions complex experienced a brief resurgence in 1926 after an investor backed the construction of the Atlantic City Speedway there. /T_he revival was short-lived as interest quickly waned among the visiting public and other investors.\nLittle remains of Amatol today. After a hike \nthrough the Pine Barrens, remnants of the munitions plant, town, and racetrack can be found at the former site of Amatol. Of the many buildings and homes constructed, only two still remain: the now- abandoned State Police barracks outside of Hammonton and one house, which was moved to the White Horse Pike, where it is still occupied.\n30 During the 1980s, the \nAmatol site was considered for use as a land/f_ill, but the proposal was rejected due to environmental concerns and opposition by local residents.\n31 Recently, in 2017, \nmore than 500 acres of Amatol (now the Pine Barrens) were preserved.\n32\nA

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [11]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [12]:
print(response)

The "best" way to write a loop in Python often depends on the specific task at hand, but there are a few common patterns you can follow. Here are some popular ways to write loops in Python:

1. **Using a `for` loop**: This is ideal for iterating over a sequence (like a list, tuple, or string).

   ```python
   # Example: Iterating over a list
   numbers = [1, 2, 3, 4, 5]
   for number in numbers:
       print(number)
   ```

2. **Using a `while` loop**: This is best when you want to repeat an action until a certain condition is no longer true.

   ```python
   # Example: Using a while loop
   count = 0
   while count < 5:
       print(count)
       count += 1
   ```

3. **Using list comprehensions**: For simple loops that generate lists, list comprehensions can be concise and readable.

   ```python
   # Example: Creating a list of squares
   squares = [x**2 for x in range(10)]
   print(squares)
   ```

4. **Using the `enumerate()` function**: If you need both the index and the value f

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [13]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [14]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [ ]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: The 'Michael Eisner Memorial Weak Executive Problem' refers to the tendency of a CEO or startup founder to hire someone who is weak in a function they themselves were once proficient in. This scenario often arises from the CEO's difficulty in letting go of the function that was their strength, leading them to choose a weaker executive for that role so they can continue to maintain influence ("be 'the man'"). The context specifically mentions Michael Eisner, the former CEO of Disney, who previously had a successful career as a TV network executive. When he acquired ABC, it performed poorly under his leadership, and his response indicated a belief that he could turn things around if he had more time, despite the reality that he was not able to manage it effectively ("guess what, he didn’t have an extra two days a week"). This example illustrates the problem of selecting weaker executives to maintain control in familiar areas rather than hiring strong leaders for those roles.

C

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

### Activity

I refactored text_utils.py to have a `BaseFileLoader` class that other file loaders inherit from. There are specific loaders for file types of `txt`, `pdf`, `docx`, and `doc`. I also added a `UniversalFileLoader` to support easy registration of new file types, making the system more extensible and modular.

<img src="./images/custom_rag_diagram.png" alt="OpenAI Logo" width="25%"/>


In [38]:
vector_db.search_by_text("Where is Amatol located?", k=25)

[(' the population of Amatol had virtually van-ished, with only a few hundred remaining. /T_he former munitions complex experienced a brief resurgence in 1926 after an investor backed the construction of the Atlantic City Speedway there. /T_he revival was short-lived as interest quickly waned among the visiting public and other investors.\nLittle remains of Amatol today. After a hike \nthrough the Pine Barrens, remnants of the munitions plant, town, and racetrack can be found at the former site of Amatol. Of the many buildings and homes constructed, only two still remain: the now- abandoned State Police barracks outside of Hammonton and one house, which was moved to the White Horse Pike, where it is still occupied.\n30 During the 1980s, the \nAmatol site was considered for use as a land/f_ill, but the proposal was rejected due to environmental concerns and opposition by local residents.\n31 Recently, in 2017, \nmore than 500 acres of Amatol (now the Pine Barrens) were preserved.\n32\nA

In [36]:
vector_db.search_by_text("What was Amatol's role in World War II?", k=3)

[('.  By 1923, Amatol Arsenal ceased operations.\nCamp Amatol\n\nIn October 1918, 2,400 troops arrived at Amatol to assist in operations there.  Their job was to bring the output to full loading capacity and they would even perform civilian tasks. In an October 14 article, many of these soldiers decided to take French leave—an unauthorized departure—by traveling to Atlantic City. A few of the soldiers were jailed when they were unable to show “leave of absence.”  By the time of the armistice, there were 3,800 officers and enlisted men at Camp Amatol, as it became known. \n\nThere were also Camp Amatol sports teams. Basketball was a big sport played amongst soldiers, including officers. A December 6, 1918 article records, “the Hammonton five defeated the U.S. Ordnance quintet from Camp Amatol by the score of 19 to 10.”  A December 13, 1918 article describes how the U.S. Ordnance Officers team defeated the Amatol team in basketball by the score of 25 to 10.  \n\nConclusion\n\nIn just a m

In [ ]:
vector_db.search_by_text("What was Amatol's role in World War II?", k=3)

In [40]:
vector_db.search_by_text("What is the history of Amatol?", k=3)

[('Town of Amatol, New Jersey, \ncommonly known as the “Amatol book,” Victor F. Hammel records a description of the construction In total, 1,600 acres had to be cleared for the plant, and 38 acres had to be cleared and 255 acres trimmed for the town.\n8 /T_his was no easy task. /T_he land was mostly \nwooded with some acres of swamp and was diﬃcult to clear due to a “dense, tangled nature of growth.”\n9 /T_he \ninitial workforce included forty men, and, within the /f_irst week, the /f_irst bunkhouse and mess hall were built.\n10\nDevelopment was rapid. By June, the population was \nnearly 2,000, and train service was added.11 By August, \nadvertisements for Amatol described it asa city with all modern improvements . . . with \nelectric lights, a /f_ire department, a police depart-ment, a modern sewage system, a modern water plant, a theatre, a Y.M.C.A, an auditorium, bowl-ing alleys, and other city amenities.\n12\nAmatol also promised many employment opportunities, \nnot just with the 

In [17]:
def ask_rag_question(question, k=3, response_length="comprehensive", include_warnings=True, confidence_required=True):
    """
    Helper to run a RAG pipeline query and print the response, context count, and similarity scores.
    """
    result = rag_pipeline.run_pipeline(
        question,
        k=k,
        response_length=response_length,
        include_warnings=include_warnings,
        confidence_required=confidence_required
    )
    print(f"Response: {result['response']}\n")
    print(f"Context Count: {result['context_count']}")
    print(f"Similarity Scores: {result['similarity_scores']}\n")

In [39]:
ask_rag_question("What is the history of Amatol", k=3)

Response: The history of Amatol begins in December 1917 with the incorporation of the Atlantic Loading Company. This company was contracted by the United States Government to construct a large shell-loading munitions plant and a corresponding town to accommodate its workers. The development of Amatol was remarkably swift, taking only nine months to complete the town and factory, ultimately reaching a population of around 7,000. 

Initially, in 1917, a total of 1,600 acres had to be cleared for the plant, while an additional 38 acres were cleared and 255 acres trimmed for the town. The land was predominantly wooded and swampy, making the clearing process quite challenging due to the dense growth. By the first week of construction, an initial workforce of forty men had already built the first bunkhouse and mess hall.

By June 1918, the population had nearly reached 2,000, prompting the addition of train services to the area. By August of the same year, advertisements promoted Amatol as a

In [41]:
ask_rag_question("What is the history of Amatol", k=25)

Response: The history of Amatol begins in December 1917 with the incorporation of the Atlantic Loading Company, which was contracted by the United States Government to construct a large shell-loading munitions plant and a town to accommodate its workers. The site chosen for this development was a tract of 6,000 acres located between East Hammonton and Elwood, New Jersey. Concerns regarding water pollution at an initially considered site near Camp Dix led to the selection of Amatol.

Construction commenced on March 4, 1918, and progressed rapidly, resulting in the completion of the town and plant within just nine months. The population quickly grew, reaching approximately 7,000 by the time operations began in July 1918, where loading operations officially started on July 31 of the same year. The munitions plant was capable of producing 60,000 shells per day among other munitions and loaded over 9,000,000 rounds of ammunition during its operational lifespan.

Amatol benefited from variou

In [37]:
ask_rag_question("Where is Amatol located?", k=25)

Response: Amatol is located in Atlantic County, New Jersey, specifically between East Hammonton and Elwood. The site of Amatol once occupied a tract of 6,000 acres and was developed as a planned shell-loading munitions plant and workers’ village during World War I (Source 3).

Context Count: 25
Similarity Scores: ['Source 1: 0.598', 'Source 2: 0.586', 'Source 3: 0.564', 'Source 4: 0.559', 'Source 5: 0.557', 'Source 6: 0.544', 'Source 7: 0.504', 'Source 8: 0.504', 'Source 9: 0.500', 'Source 10: 0.499', 'Source 11: 0.493', 'Source 12: 0.489', 'Source 13: 0.482', 'Source 14: 0.479', 'Source 15: 0.469', 'Source 16: 0.467', 'Source 17: 0.459', 'Source 18: 0.456', 'Source 19: 0.448', 'Source 20: 0.443', 'Source 21: 0.412', 'Source 22: 0.410', 'Source 23: 0.395', 'Source 24: 0.393', 'Source 25: 0.387']



In [18]:
ask_rag_question("What was Amatol's role in World War I?")

Response: Amatol served as a planned shell-loading munitions plant and workers' village during World War I. Its construction began in December 1917, and it was built to support the war effort by providing the necessary facilities for loading munitions. By the time it reached completion, it had a population of around 7,000 and was instrumental in expanding the industrial economic base of Atlantic County due to America's entry into the war on April 6, 1917. The site focused on maximizing output, as evident from the arrival of troops in October 1918, who were tasked with bringing the output to full loading capacity and performing civilian tasks (Source 3).

Context Count: 3
Similarity Scores: ['Source 1: 0.568', 'Source 2: 0.543', 'Source 3: 0.520']



In [19]:
ask_rag_question("When did Amatol become a ghost town?")

Response: Amatol became a ghost town shortly after the signing of the Armistice in World War I, as "the population of Amatol had virtually vanished, with only a few hundred remaining there" in just a matter of months post-Armistice. The decline in population is associated with the early 1920s, as the town experienced a brief resurgence in 1926 but soon again faced diminished interest, leading to its eventual abandonment.

Context Count: 3
Similarity Scores: ['Source 1: 0.616', 'Source 2: 0.585', 'Source 3: 0.573']



In [20]:
ask_rag_question("Can you tell me about the Amatol raceway?")

Response: The Amatol raceway, referred to as the Atlantic City Speedway, experienced a brief resurgence in 1926 when it was constructed with the backing of an investor. However, this revival was short-lived as interest quickly waned among the public and other investors, leading to a decline in its popularity. Today, little remains of the Amatol site, although remnants of the racetrack can still be found along with those of the munitions plant and town after a hike through the Pine Barrens.

Context Count: 3
Similarity Scores: ['Source 1: 0.592', 'Source 2: 0.569', 'Source 3: 0.501']



In [23]:
ask_rag_question("What was Amatol's role in World War II?", k=100)

Response: I don't know.

Context Count: 100
Similarity Scores: ['Source 1: 0.564', 'Source 2: 0.549', 'Source 3: 0.521', 'Source 4: 0.520', 'Source 5: 0.511', 'Source 6: 0.508', 'Source 7: 0.508', 'Source 8: 0.504', 'Source 9: 0.503', 'Source 10: 0.491', 'Source 11: 0.489', 'Source 12: 0.473', 'Source 13: 0.472', 'Source 14: 0.471', 'Source 15: 0.470', 'Source 16: 0.462', 'Source 17: 0.445', 'Source 18: 0.424', 'Source 19: 0.415', 'Source 20: 0.415', 'Source 21: 0.414', 'Source 22: 0.406', 'Source 23: 0.399', 'Source 24: 0.396', 'Source 25: 0.379', 'Source 26: 0.363', 'Source 27: 0.363', 'Source 28: 0.355', 'Source 29: 0.349', 'Source 30: 0.341', 'Source 31: 0.295', 'Source 32: 0.241', 'Source 33: 0.200', 'Source 34: 0.200', 'Source 35: 0.194', 'Source 36: 0.192', 'Source 37: 0.186', 'Source 38: 0.172', 'Source 39: 0.163', 'Source 40: 0.157', 'Source 41: 0.156', 'Source 42: 0.153', 'Source 43: 0.152', 'Source 44: 0.152', 'Source 45: 0.151', 'Source 46: 0.151', 'Source 47: 0.151', 'Sour

In [22]:
ask_rag_question("What is the infinity of life?")

Response: The infinity of life, as described in the context, is a state where everything is perfect, whole, and complete, yet life is constantly changing. It emphasizes that there is no beginning and no end but rather a continuous cycling and recycling of substances and experiences. The passage reflects a belief that life is never stuck, static, or stale, as each moment is always new and fresh. This perspective suggests a deep connection with a creative power, asserting that individuals have the ability to create their own circumstances and that every moment represents a new beginning (Source 1).

Context Count: 3
Similarity Scores: ['Source 1: 0.421', 'Source 2: 0.351', 'Source 3: 0.299']



In [23]:
ask_rag_question("What did Rick Ruben say?")

Response: Rick Ruben stated, "Tools will come. Tools will go. Vibe coding is here." This suggests an emphasis on the enduring nature of "vibe coding" despite the transitory nature of tools.

Context Count: 3
Similarity Scores: ['Source 1: 0.455', 'Source 2: 0.282', 'Source 3: 0.267']

